In [1]:
# !pip install tqdm
# !pip install beautifulsoup4

In [28]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import os
import time
import numpy as np


## initialization
print(time.ctime(time.time()))
if not os.path.exists("dataset"):
    os.mkdir("dataset")
    
data = pd.DataFrame({"page": [],
                    "name": [],
                    "general_name": [],
                    "drug_type": [],
                    "consumption": [],
                    "license": [],
                    "brand": [],
                    "country": [],
                    "producer": [],
                    "license_exp": [],
                    "price_per_pack": [],
                    "unit_price": [],
                    "GTIN": [],
                    "IRC": [],
                    "number_per_pack": [],
                    "ingredients": [],
                    "ATC_code": []})

## change numbers base on your system and website
start = 1
max_allowed_i = 1000    # 45433
per_partition = 1000    # 5000
num_features_to_crawl = 16
break_lim = 5


## crawler and organizer 
breaker = 0
for i in tqdm(range(start, max_allowed_i+1)):
    temp_data = [None] * (num_features_to_crawl + 1)
    res = requests.get(f"https://irc.fda.gov.ir/nfi/Detail/{i}")
    soup = BeautifulSoup(res.text, 'html.parser')
    
    # tags
    tag0 = soup.select("""body > div > div.animationPage > div.MainBoxSearchDetail > div.paddingSearchDetail > div > div > div.col-lg-12.col-md-12.col-sm-12.col-xs-12.padding0 > div > div.col-lg-5.col-md-5.col-sm-6.col-xs-12""")
    tag1 = soup.select("""body > div > div.animationPage > div.MainBoxSearchDetail > div.paddingSearchDetail > div > div > div.col-lg-12.col-md-12.col-sm-12.col-xs-12.padding0 > div > div""")
    tag2 = soup.select("""body > div > div.animationPage > div.MainBoxSearchDetail > div.paddingSearchDetail > div > div > div.col-lg-12.col-md-12.col-sm-12.col-xs-12.padding0 > div > div > div > div.col-lg-7.col-md-12.col-sm-12.col-xs-12 > div > div.col-lg-5.col-md-5.col-sm-12.col-xs-12 > span""")
    tag3 = soup.select("""body > div > div.animationPage > div.MainBoxSearchDetail > div.paddingSearchDetail > div > div > div.col-lg-12.col-md-12.col-sm-12.col-xs-12.padding0 > div > div > div > div.col-lg-5.col-md-12.col-sm-12.col-xs-12 > div.col-lg-12.col-md-12.col-sm-12.col-xs-12.boldtext.txtEnglish-ltr > label""")

    try:
        # organizing data
        temp_data[0] = i
        temp_data[1] = tag0[0].contents[3].string.strip()
        temp_data[2] = tag1[1].contents[3].text.strip()
        temp_data[3] = tag0[1].contents[3].string.strip()
        temp_data[4] = tag0[2].contents[3].string.strip()
        temp_data[5] = tag0[3].contents[3].string.strip()
        temp_data[6] = tag1[5].contents[3].text.strip()
        temp_data[7] = tag1[5].contents[3].img.get('title')
        temp_data[8] = tag0[5].contents[3].string.strip()
        temp_data[9] = tag0[6].contents[3].string.strip()
        temp_data[10] = tag0[7].contents[3].string.strip()
        temp_data[11] = tag1[9].contents[3].text.strip()
        temp_data[12] = tag0[8].contents[3].string.strip()
        temp_data[13] = tag0[9].contents[3].string.strip()
        temp_data[14] = tag0[10].contents[3].string.strip()
        temp_data[15] = tag2[0].string.strip()
        temp_data[16] = tag3[0].string
        
        breaker = 0   # if data seen
    
    except IndexError: # if see any error delete "IndexError"
        temp_data[0] = i
        temp_data[1:] = [np.nan] * num_features_to_crawl
        breaker += 1

        if breaker == break_lim:
            data.to_csv(f"dataset/data{i}.csv", index=False)
            break
    
    
    ## data saver
    data.loc[i] = temp_data
    if i == max_allowed_i:
        data.to_csv(f"dataset/data{i}.csv", index=False)
        # print(f"{i}/{max_allowed_i}")
        break
    elif i % per_partition == 0:
        data.to_csv(f"dataset/data{i}.csv", index=False)
        data = pd.DataFrame({"page": [],
                             "name": [],
                    "general_name": [],
                    "drug_type": [],
                    "consumption": [],
                    "license": [],
                    "brand": [],
                    "country": [],
                    "producer": [],
                    "license_exp": [],
                    "price_per_pack": [],
                    "unit_price": [],
                    "GTIN": [],
                    "IRC": [],
                    "number_per_pack": [],
                    "ingredients": [],
                    "ATC_code": []})


print(time.ctime(time.time()))

Fri Jan 26 23:57:06 2024


100%|████████████████████████████████████████████████▉| 999/1000 [12:22<00:00,  1.35it/s]

Sat Jan 27 00:09:28 2024
